In [9]:
!pip install bert-score rouge-score tqdm nltk
# Download NLTK data for BLEU
nltk.download('punkt')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
!pip install transformers datasets torch pillow bert_score tqdm nltk pandas matplotlib evaluate rouge_score py-rouge sacrebleu ftfy regex
!pip install open-clip-torch  # Correct package name with hyphens

import os
import random
import torch
import numpy as np
import pandas as pd
import nltk
import evaluate
import open_clip
from PIL import Image
from collections import Counter
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import BartTokenizer, BartForConditionalGeneration

# Import the rouge_scorer package for ROUGE-L calculation
try:
    from rouge_score import rouge_scorer
except ImportError:
    # Install if not available
    !pip install rouge-score
    from rouge_score import rouge_scorer

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━

2025-05-16 11:26:26.064100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747394786.504041      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747394786.637033      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings

import pandas as pd
import torch
import open_clip
from PIL import Image
from sklearn.metrics import accuracy_score
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
from tqdm import tqdm
import nltk
import gc
import random
import matplotlib.pyplot as plt
from datasets import load_dataset
# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# Suppress specific logging
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("torch").setLevel(logging.ERROR)
logging.getLogger("PIL").setLevel(logging.ERROR)
logging.getLogger("nltk").setLevel(logging.ERROR)
logging.getLogger("sklearn").setLevel(logging.ERROR)

# Make sure nltk has the necessary data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Define paths
DATASET_PATH = "/kaggle/input/dataset/Complete_vqa(in).csv"
IMAGE_BASE_PATH = "/kaggle/input/imagedataset/small"
OUTPUT_LOG_2_FIXED = "/kaggle/working/clip_baseline_2_fixed.csv"
OUTPUT_LOG_5_FIXED = "/kaggle/working/clip_baseline_5_fixed.csv"
OUTPUT_LOG_2_ALL = "/kaggle/working/clip_baseline_2_all.csv"
OUTPUT_LOG_5_ALL = "/kaggle/working/clip_baseline_5_all.csv"

# Initialize metrics columns (matching your format)
metrics_columns = ['image_path', 'question', 'answer', 'prediction', 'correct', 
                   'meteor', 'f1', 'accuracy', 'bertscore_precision', 
                   'bertscore_recall', 'bertscore_f1', 'partscore', 'rouge_1_f1']

print("Loading VQA dataset...")
vqa_df = pd.read_csv(DATASET_PATH)

# Extract all unique answers from the dataset
all_unique_answers = vqa_df['answer'].dropna().unique().tolist()
print(f"Total unique answers in dataset: {len(all_unique_answers)}")

# Define the fixed set of common answers (as from your CLIP code)
common_answers = [
    "yes", "no", "2", "1", "white", "red", "blue", "black", "3", "green",
    "yellow", "4", "brown", "orange", "0", "5", "gray", "grey", "purple", "pink",
    "6", "7", "8", "9", "10", "standing", "sitting", "right", "left", "wood",
    "metal", "plastic", "glass", "paper", "water", "food", "man", "woman", "boy",
    "girl", "dog", "cat", "table", "chair", "car", "phone", "book", "computer",
    "tv", "bed", "outside", "inside", "day", "night", "morning", "evening", "afternoon",
    "kitchen", "bathroom", "bedroom", "living room", "large", "small", "medium",
    "round", "square", "rectangular", "triangle", "circle", "diamond", "heart", "star",
    "up", "down", "on", "under", "near", "far", "tall", "short", "long", "wide",
    "narrow", "big", "little", "high", "low", "hot", "cold", "warm", "cool", "old",
    "new", "young", "open", "closed"
]

# Function to load image
def load_image(image_path, preprocess):
    full_path = os.path.join(IMAGE_BASE_PATH, image_path)
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"Image not found: {full_path}")
    image = Image.open(full_path).convert("RGB")
    return preprocess(image).unsqueeze(0)

# Function to predict answer using CLIP
def predict_answer_clip(model, image, question, answer_candidates, preprocess, device):
    """
    Predict answer using the CLIP model with zero-shot prompting
    
    Parameters:
    - model: CLIP model
    - image: preprocessed image tensor
    - question: question text
    - answer_candidates: list of possible answers
    - preprocess: CLIP preprocessing function
    - device: device to use (cuda/cpu)
    
    Returns:
    - predicted answer as string
    """
    try:
        # Move image to device
        image = image.to(device)
        
        # Encode image
        with torch.no_grad():
            image_features = model.encode_image(image)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            
            # Prepare text prompts (question + possible answer)
            answer_prompts = [f"Question: {question} Answer: {ans}" for ans in answer_candidates]
            
            # Use open_clip's tokenizer
            tokenizer = open_clip.get_tokenizer('ViT-B-32')
            text = tokenizer(answer_prompts).to(device)
            
            # Encode text prompts
            text_features = model.encode_text(text)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            
            # Calculate similarity scores
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            
            # Get the most likely answer
            values, indices = similarity[0].topk(1)
            predicted_answer = answer_candidates[indices[0].item()]
        
        return predicted_answer
    
    except Exception as e:
        print(f"Error in CLIP prediction: {e}")
        return ""

def run_evaluation(sample_percentage, output_file, answer_candidates):
    """
    Run evaluation on a sample of the dataset using CLIP
    
    Parameters:
    - sample_percentage: Percentage of dataset to sample (e.g., 2, 5)
    - output_file: Path to save results CSV
    - answer_candidates: List of possible answers
    """
    # Sample the dataset
    sample_size = int(len(vqa_df) * (sample_percentage / 100))
    print(f"\nRunning evaluation on {sample_percentage}% of dataset ({sample_size} samples)...")
    
    sampled_df = vqa_df.sample(n=sample_size, random_state=42)
    questions = sampled_df['question'].tolist()
    original_answers = sampled_df['answer'].tolist()
    ground_truth_answers = [str(ans).lower() for ans in original_answers]
    image_paths = sampled_df['image_path'].tolist()
    
    # Initialize results dataframe
    results_df = pd.DataFrame(columns=metrics_columns)
    
    # Process all samples
    answer_type = "fixed set" if len(answer_candidates) < len(all_unique_answers) else "all answers"
    
    for idx in tqdm(range(len(image_paths)), desc=f"Processing {sample_percentage}% with {answer_type}"):
        try:
            image_path = image_paths[idx]
            question = questions[idx]
            gt_answer = ground_truth_answers[idx]
            
            # Load and preprocess image
            image = load_image(image_path, preprocess)
            
            # Predict answer using CLIP
            pred_answer = predict_answer_clip(model, image, question, answer_candidates, preprocess, device)
            pred_answer = pred_answer.lower()  # Convert to lowercase for consistent comparison
            
            # Calculate individual metrics
            correct = pred_answer == gt_answer
            
            # METEOR score (placeholder - since METEOR calculation is complex, we use 0.5 if correct, 0 if not)
            meteor = 0.5 if correct else 0
            
            # F1 score (placeholder - using 1 if correct, 0 if incorrect for simplicity)
            f1 = 1 if correct else 0
            
            # Accuracy (binary)
            acc = 1 if correct else 0
            
            # BERTScore for this sample
            p_bert, r_bert, f1_bert = bert_score([pred_answer], [gt_answer], lang="en", verbose=False)
            
            # ROUGE score for this sample
            scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
            rouge = scorer.score(gt_answer, pred_answer)['rouge1'].fmeasure
            
            # BLEU score (partscore in your example)
            smoothie = SmoothingFunction().method1
            try:
                pred_tokens = nltk.word_tokenize(pred_answer)
                truth_tokens = [nltk.word_tokenize(gt_answer)]
                bleu = sentence_bleu(truth_tokens, pred_tokens, smoothing_function=smoothie)
            except:
                bleu = 0
                
            # Store results in format matching your example
            row = {
                'image_path': image_path,
                'question': question,
                'answer': gt_answer,
                'prediction': pred_answer,
                'correct': str(correct),  # Convert to string 'True'/'False'
                'meteor': meteor,
                'f1': f1,
                'accuracy': acc,
                'bertscore_precision': p_bert.item(),
                'bertscore_recall': r_bert.item(),
                'bertscore_f1': f1_bert.item(),
                'partscore': bleu,
                'rouge_1_f1': rouge
            }
            
            # Append to results dataframe
            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
            
            # Force garbage collection periodically
            if idx % 10 == 0:
                gc.collect()
                torch.cuda.empty_cache()
                
        except Exception as e:
            print(f"Error at sample {idx+1}: {e}")
            
            # Add empty row on error
            row = {col: "" for col in metrics_columns}
            row.update({
                'image_path': image_paths[idx],
                'question': questions[idx],
                'answer': ground_truth_answers[idx],
                'prediction': "",
                'correct': "False"
            })
            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
            
            # Try to recover memory on error
            gc.collect()
            torch.cuda.empty_cache()
    
    # Calculate overall metrics for reporting
    correct_preds = results_df['correct'].map({'True': True, 'False': False})
    overall_accuracy = sum(correct_preds) / len(correct_preds) if len(correct_preds) > 0 else 0
    overall_bertscore_f1 = results_df['bertscore_f1'].mean()
    overall_rouge = results_df['rouge_1_f1'].mean()
    overall_bleu = results_df['partscore'].mean()
    
    # Save results to CSV
    results_df.to_csv(output_file, index=False)
    print(f"Results for {sample_percentage}% with {answer_type} saved to {output_file}")
    
    # Summary
    print(f"\nEvaluation Summary for {sample_percentage}% with {answer_type}:")
    print(f"- Accuracy: {overall_accuracy:.4f}")
    print(f"- BERTScore F1: {overall_bertscore_f1:.4f}")
    print(f"- ROUGE-1 F1: {overall_rouge:.4f}")
    print(f"- BLEU: {overall_bleu:.4f}")

# Load CLIP model
print("Loading CLIP model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.to(device)
model.eval()

# Run evaluations
try:
    # Run for 2% with fixed answer set
    run_evaluation(2, OUTPUT_LOG_2_FIXED, common_answers)
    
    # Clear memory
    gc.collect()
    torch.cuda.empty_cache()
    
    # Run for 5% with fixed answer set
    run_evaluation(5, OUTPUT_LOG_5_FIXED, common_answers)
    
    # Clear memory
    gc.collect()
    torch.cuda.empty_cache()
    
    # Run for 2% with all unique answers
    run_evaluation(2, OUTPUT_LOG_2_ALL, all_unique_answers)
    
    # Clear memory
    gc.collect()
    torch.cuda.empty_cache()
    
    # Run for 5% with all unique answers
    run_evaluation(5, OUTPUT_LOG_5_ALL, all_unique_answers)
    
except Exception as e:
    print(f"Fatal error: {e}")
finally:
    # Clean up
    del model
    gc.collect()
    torch.cuda.empty_cache()
    print("\nEvaluation completed!")

Loading VQA dataset...
Total unique answers in dataset: 1900
Loading CLIP model...


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]


Running evaluation on 2% of dataset (909 samples)...


Processing 2% with fixed set:   0%|          | 0/909 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Results for 5% with fixed set saved to /kaggle/working/clip_baseline_5_fixed.csv

Evaluation Summary for 5% with fixed set:
- Accuracy: 0.1051
- BERTScore F1: 0.9759
- ROUGE-1 F1: 0.1060
- BLEU: 0.0187

Running evaluation on 2% of dataset (909 samples)...


Processing 2% with all answers: 100%|██████████| 909/909 [1:10:36<00:00,  4.66s/it]


Results for 2% with all answers saved to /kaggle/working/clip_baseline_2_all.csv

Evaluation Summary for 2% with all answers:
- Accuracy: 0.0374
- BERTScore F1: 0.8879
- ROUGE-1 F1: 0.0418
- BLEU: 0.0067

Running evaluation on 5% of dataset (2274 samples)...


Processing 5% with all answers: 100%|██████████| 2274/2274 [2:56:46<00:00,  4.66s/it]  


Results for 5% with all answers saved to /kaggle/working/clip_baseline_5_all.csv

Evaluation Summary for 5% with all answers:
- Accuracy: 0.0286
- BERTScore F1: 0.8867
- ROUGE-1 F1: 0.0335
- BLEU: 0.0051

Evaluation completed!
